In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
H1

SHAP 분석을 통한 5개 지표군의 feature importance 계산

목표:
1. XGBoost 모델로 고위험/정상 분류
2. SHAP으로 각 변수의 중요도 계산
3. 지표군별 영향력 시각화
4. T-검정 결과와 비교
"""

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("가설 1: SHAP 분석을 통한 변수 중요도 계산")
print("="*80)

# ============================================================================
# 1. 데이터 로드
# ============================================================================
print("\n[1단계] 데이터 로드")

result_dir = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석"
data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv"

# 클러스터링 결과
cluster_result = pd.read_csv(f"{result_dir}/클러스터링_결과_완전판.csv")

# 원본 데이터
df_full = pd.read_csv(data_path)

# 병합
df = cluster_result.merge(df_full, on='가맹점구분번호', how='left')

print(f"데이터: {len(df):,}개")

# ============================================================================
# 2. 5개 지표군 변수 정의
# ============================================================================
print("\n" + "="*80)
print("[2단계] 5개 지표군 변수 정의")
print("="*80)

# 운영지표
operating_indicators = [
    '가맹점 운영개월수 구간', '매출금액 구간', '매출건수 구간',
    '객단가 구간', '취소율 구간', '배달매출금액 비율',
    '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액',
    '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
    '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액'
]

# 고객지표
customer_indicators = [
    '유니크 고객 수 구간', '재방문 고객 비중', '신규 고객 비중',
    '거주 이용 고객 비율', '직장 이용 고객 비율', '유동인구 이용 고객 비율',
    '남성 20대이하 고객 비중', '남성 30대 고객 비중', '남성 40대 고객 비중', '남성 50대 고객 비중', '남성 60대이상 고객 비중',
    '여성 20대이하 고객 비중', '여성 30대 고객 비중', '여성 40대 고객 비중', '여성 50대 고객 비중', '여성 60대이상 고객 비중'
]

# 경쟁밀집지표
competition_indicators = [
    '동일 업종 매출금액 비율', '동일 업종 매출건수 비율',
    '동일 업종 내 매출 순위 비율', '동일 상권 내 매출 순위 비율',
    '동일 업종 내 해지 가맹점 비중', '동일 상권 내 해지 가맹점 비중'
]

# 접근성지표
accessibility_indicators = [
    '1km내_버스정류장수', '1km내_총버스승하차', '1km내_지하철역수', '1km내_총지하철승하차',
    '500m내_버스정류장수', '500m내_평균버스승하차', '500m내_지하철역수', '500m내_평균지하철승하차'
]

# 경제환경지표
economic_indicators = [
    '생활물가지수', '식품', '식품 이외', '전월세', '전·월세포함 생활물가지수',
    '지출_총금액', '식료품_지출_총금액', '여가_지출_총금액'
]

# 실제 존재하는 컬럼만 필터링
def filter_existing_cols(indicator_list, df_cols):
    return [col for col in indicator_list if col in df_cols]

operating_cols = filter_existing_cols(operating_indicators, df.columns)
customer_cols = filter_existing_cols(customer_indicators, df.columns)
competition_cols = filter_existing_cols(competition_indicators, df.columns)
accessibility_cols = filter_existing_cols(accessibility_indicators, df.columns)
economic_cols = filter_existing_cols(economic_indicators, df.columns)

all_features = operating_cols + customer_cols + competition_cols + accessibility_cols + economic_cols

print(f"\n운영지표: {len(operating_cols)}개")
print(f"고객지표: {len(customer_cols)}개")
print(f"경쟁밀집지표: {len(competition_cols)}개")
print(f"접근성지표: {len(accessibility_cols)}개")
print(f"경제환경지표: {len(economic_cols)}개")
print(f"총 변수: {len(all_features)}개")

# ============================================================================
# 3. 타겟 변수 정의 (고위험 vs 정상)
# ============================================================================
print("\n" + "="*80)
print("[3단계] 타겟 변수 정의")
print("="*80)

# 고위험: 클러스터 0, 2 → 1
# 정상: 클러스터 1 → 0
df['is_high_risk'] = df['cluster'].apply(lambda x: 1 if x in [0, 2] else 0)

print(f"\n고위험: {(df['is_high_risk'] == 1).sum():,}개")
print(f"정상: {(df['is_high_risk'] == 0).sum():,}개")

# ============================================================================
# 4. 데이터 전처리
# ============================================================================
print("\n" + "="*80)
print("[4단계] 데이터 전처리")
print("="*80)

# 수치형 변수만 선택
numeric_features = []
for col in all_features:
    if pd.api.types.is_numeric_dtype(df[col]):
        numeric_features.append(col)

print(f"\n수치형 변수: {len(numeric_features)}개")

# 결측치 처리 (중앙값으로 대체)
X = df[numeric_features].copy()
X = X.fillna(X.median())

y = df['is_high_risk']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# ============================================================================
# 5. XGBoost 모델 학습
# ============================================================================
print("\n" + "="*80)
print("[5단계] XGBoost 모델 학습")
print("="*80)

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain: {len(X_train):,}개")
print(f"Test: {len(X_test):,}개")

# XGBoost 모델
model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)

print("\n모델 학습 중...")
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# 성능 평가
print("\n[모델 성능]")
print(classification_report(y_test, y_pred, target_names=['정상', '고위험']))
print(f"\nROC-AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

# ============================================================================
# 6. SHAP 분석
# ============================================================================
print("\n" + "="*80)
print("[6단계] SHAP 분석")
print("="*80)

import shap

# SHAP TreeExplainer
print("\nSHAP Explainer 생성 중...")
explainer = shap.TreeExplainer(model)

print("SHAP values 계산 중... (시간 소요됨)")
# 전체 데이터는 너무 크므로 샘플링
sample_size = min(5000, len(X_test))
X_sample = X_test.sample(n=sample_size, random_state=42)
shap_values = explainer.shap_values(X_sample)

print(f"✓ SHAP values 계산 완료 ({sample_size}개 샘플)")

# ============================================================================
# 7. Feature Importance (SHAP 기반)
# ============================================================================
print("\n" + "="*80)
print("[7단계] Feature Importance (SHAP)")
print("="*80)

# 절댓값 평균으로 중요도 계산
shap_importance = pd.DataFrame({
    '변수': X_sample.columns,
    'SHAP_importance': np.abs(shap_values).mean(axis=0)
})

shap_importance = shap_importance.sort_values('SHAP_importance', ascending=False)

print("\n[TOP 20 중요 변수 (SHAP)]")
print(shap_importance.head(20).to_string(index=False))

# ============================================================================
# 8. 지표군별 SHAP Importance
# ============================================================================
print("\n" + "="*80)
print("[8단계] 지표군별 SHAP Importance")
print("="*80)

# 지표군 매핑
def get_indicator_group(col):
    if col in operating_cols:
        return '운영지표'
    elif col in customer_cols:
        return '고객지표'
    elif col in competition_cols:
        return '경쟁밀집지표'
    elif col in accessibility_cols:
        return '접근성지표'
    elif col in economic_cols:
        return '경제환경지표'
    else:
        return '기타'

shap_importance['지표군'] = shap_importance['변수'].apply(get_indicator_group)

# 지표군별 평균 중요도
indicator_importance = shap_importance.groupby('지표군')['SHAP_importance'].agg([
    ('평균_SHAP', 'mean'),
    ('합계_SHAP', 'sum'),
    ('변수수', 'count')
]).sort_values('평균_SHAP', ascending=False)

print("\n[지표군별 SHAP Importance]")
print(indicator_importance)

# ============================================================================
# 9. T-검정 vs SHAP 비교
# ============================================================================
print("\n" + "="*80)
print("[9단계] T-검정 vs SHAP 비교")
print("="*80)

# T-검정 결과 로드
ttest_results = pd.read_csv("/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/4_인사이트/지표군별_유의변수.csv")

# SHAP과 병합
comparison = shap_importance.merge(
    ttest_results[['지표군', '변수', 'Cohens_d', 'p값', '유의성']],
    on=['지표군', '변수'],
    how='left'
)

comparison = comparison.sort_values('SHAP_importance', ascending=False)

print("\n[TOP 10: SHAP vs Cohen's d 비교]")
print(comparison[['지표군', '변수', 'SHAP_importance', 'Cohens_d', 'p값', '유의성']].head(10).to_string(index=False))

# ============================================================================
# 10. 결과 저장
# ============================================================================
print("\n" + "="*80)
print("[10단계] 결과 저장")
print("="*80)

output_dir = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/4_인사이트"

# SHAP importance 저장
shap_importance.to_csv(f"{output_dir}/SHAP_변수중요도.csv", index=False, encoding='utf-8-sig')
indicator_importance.to_csv(f"{output_dir}/SHAP_지표군별중요도.csv", encoding='utf-8-sig')
comparison.to_csv(f"{output_dir}/SHAP_vs_Ttest_비교.csv", index=False, encoding='utf-8-sig')

print("✓ SHAP_변수중요도.csv 저장")
print("✓ SHAP_지표군별중요도.csv 저장")
print("✓ SHAP_vs_Ttest_비교.csv 저장")

# ============================================================================
# 11. SHAP 시각화
# ============================================================================
print("\n" + "="*80)
print("[11단계] SHAP 시각화")
print("="*80)

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')  # 백그라운드 모드

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 1. Summary Plot
print("\n1) Summary Plot 생성 중...")
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_sample, show=False, max_display=20)
plt.title('SHAP Summary Plot (TOP 20 변수)', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig(f"{output_dir}/SHAP_summary_plot.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ SHAP_summary_plot.png 저장")

# 2. Bar Plot
print("2) Bar Plot 생성 중...")
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False, max_display=20)
plt.title('SHAP Feature Importance (TOP 20)', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig(f"{output_dir}/SHAP_bar_plot.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ SHAP_bar_plot.png 저장")

# 3. 지표군별 중요도 막대그래프
print("3) 지표군별 중요도 그래프 생성 중...")
fig, ax = plt.subplots(figsize=(10, 6))
indicator_importance['평균_SHAP'].plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('평균 SHAP Importance', fontsize=12)
ax.set_ylabel('지표군', fontsize=12)
ax.set_title('지표군별 SHAP Importance', fontsize=16, pad=20)
ax.invert_yaxis()
plt.tight_layout()
plt.savefig(f"{output_dir}/SHAP_지표군별중요도.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ SHAP_지표군별중요도.png 저장")

print("\n" + "="*80)
print("SHAP 분석 완료!")
print("="*80)
print(f"✅ XGBoost ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print(f"✅ SHAP 변수 분석: {len(shap_importance)}개")
print(f"✅ 지표군별 중요도 계산 완료")
print(f"✅ 시각화 3종 생성 완료")
print("="*80)


가설 1: SHAP 분석을 통한 변수 중요도 계산

[1단계] 데이터 로드
데이터: 98,117개

[2단계] 5개 지표군 변수 정의

운영지표: 16개
고객지표: 16개
경쟁밀집지표: 6개
접근성지표: 8개
경제환경지표: 8개
총 변수: 54개

[3단계] 타겟 변수 정의

고위험: 12,344개
정상: 85,773개

[4단계] 데이터 전처리

수치형 변수: 48개
X shape: (98117, 48)
y shape: (98117,)

[5단계] XGBoost 모델 학습

Train: 78,493개
Test: 19,624개

모델 학습 중...

[모델 성능]
              precision    recall  f1-score   support

          정상       0.98      1.00      0.99     17155
         고위험       0.96      0.85      0.90      2469

    accuracy                           0.98     19624
   macro avg       0.97      0.92      0.94     19624
weighted avg       0.98      0.98      0.98     19624


ROC-AUC Score: 0.9967

[6단계] SHAP 분석

SHAP Explainer 생성 중...
SHAP values 계산 중... (시간 소요됨)
✓ SHAP values 계산 완료 (5000개 샘플)

[7단계] Feature Importance (SHAP)

[TOP 20 중요 변수 (SHAP)]
              변수  SHAP_importance
    1km내_총지하철승하차         0.381921
     1km내_버스정류장수         0.280844
   500m내_평균버스승하차         0.201374
  남성 60대이상 고객 비중         0.200571
     1